In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm  # To see Progress
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

REBUILD_DATA = False  #Only Rebuild first time (preprocess data first time)

class HappyOrSad():
    IMG_SIZE = 50   #Change size to 50X50

    HAPPY = "FaceExpression/happy"
    SAD = "FaceExpression/sad"

    LABELS = {HAPPY: 0,SAD: 1}

    training_data = []

    happyCount = 0
    sadCount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label,f)
                    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))
                    self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])

                    if label==self.SAD:
                        self.sadCount +=1
                    elif label == self.HAPPY:
                        self.happyCount+=1
                except Exception as e:
                    # pass
                    print(e)
        np.random.shuffle(self.training_data)
        np.save("training_data.npy",self.training_data)
        print("SAD  : ",self.sadCount)
        print("HAPPY  : ",self.happyCount)

class Net(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1 = nn.Conv2d(1,32,5) # 1 input image channel, 32 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv2d(32,64,5) # 32 input image channel, 64 output channels, 5x5 square convolution kernel
        self.conv3 = nn.Conv2d(64,128,5) # 64 input image channel, 128 output channels, 5x5 square convolution kernel

        x = torch.randn(50,50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear,512) #  input features, 512 output features
        self.fc2 = nn.Linear(512,2) # 512 input features, 2 output features
    
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2)) # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2)) # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2)) # Max pooling over a (2,2) window

        # print(x[0].shape)
        if self._to_linear is None:
            sh = x[0].shape
            self._to_linear = sh[0]*sh[1]*sh[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x,dim=1) # activation function
    

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

net = Net().to(device)

if REBUILD_DATA:
    happyorsad = HappyOrSad()
    happyorsad.make_training_data()       

training_data = np.load("training_data.npy",allow_pickle=True)
print(len(training_data))

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X),len(test_X))

Running on GPU
305
30
275 30


C:\Users\smit2\AppData\Local\Temp\ipykernel_15724\4075253867.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)


In [2]:
def fwd_pass(X,y,train = False):
    if train:
        net.zero_grad()
    # print(X)
    outputs = net(X)
    # print(outputs)
    matches = [torch.argmax(i)==torch.argmax(j) for i,j in zip(outputs,y)]
    # print(matches)
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs,y)

    if train:
        loss.backward()
        optimizer.step()
    return acc,loss



In [3]:
def test(size = 10):
    # Get Random 10 samples
    random_start = np.random.randint(len(test_X)-size)
    X,y = test_X[random_start:random_start+size],test_y[random_start:random_start+size]
    with torch.no_grad():
        val_acc,val_loss = fwd_pass(X.view(-1,1,50,50).to(device),y.to(device))
    return val_acc,val_loss

val_acc,val_loss = test()
print(val_acc,val_loss)

0.5 tensor(0.2491, device='cuda:0')


In [6]:
import time 

MODEL_NAME = f"MODEL - {int(time.time())}"

net = Net().to(device=device)
optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()

print(MODEL_NAME)

def train():
    BATCH_SIZE = 20
    EPOCHS = 300
    with open("model.log","a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50).to(device)
                batch_y = train_y[i:i+BATCH_SIZE].to(device)

                acc,loss = fwd_pass(train=True,X=batch_X,y=batch_y)
                if i%5 == 0:
                   val_acc, val_loss = test(size = 10)
                   f.write(f"{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)},{round(float(loss),4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n")
train()
                

MODEL - 1686747730


100%|██████████| 14/14 [00:00<00:00, 30.33it/s]
